# k-最近邻（kNN）练习

完成并提交此工作表（包括其输出和任何在工作表外的支持代码）作为你的作业提交。更多细节请参见课程网站上的[作业页面](http://vision.stanford.edu/teaching/cs231n/assignments.html)。

kNN 分类器包含两个阶段：

- 在训练阶段，分类器接收训练数据并简单地记住它
- 在测试阶段，kNN 通过与所有训练图像进行比较，并转移 k 个最相似训练样本的标签来对每个测试图像进行分类
- k 的值通过交叉验证确定

在本练习中，你将实现这些步骤，理解基本的图像分类流程、交叉验证，并熟练编写高效的向量化代码。

<details><summary>英文原文</summary>

# k-Nearest Neighbor (kNN) exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

The kNN classifier consists of two stages:

- During training, the classifier takes the training data and simply remembers it
- During testing, kNN classifies every test image by comparing to all training images and transfering the labels of the k most similar training examples
- The value of k is cross-validated

In this exercise you will implement these steps and understand the basic Image Classification pipeline, cross-validation, and gain proficiency in writing efficient, vectorized code.

</details>

In [ ]:
# 运行本笔记本的一些设置代码。

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# 这是一些魔法命令，使 matplotlib 图像在笔记本内联显示
# 而不是在新窗口中显示。
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 设置图像默认大小
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# 还有一些魔法命令，使笔记本能自动重新加载外部 python 模块；
# 参考 http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# 加载原始 CIFAR-10 数据。
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

# 清理变量以防止多次加载数据（可能导致内存问题）
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# 做一个健壮性检查，打印训练和测试数据的大小。
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# 可视化数据集中的一些样本。
# 我们展示每个类别的部分训练图像样本。
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# 对数据进行子采样，以便本练习更高效地运行代码
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# 将图像数据重塑为行向量
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from cs231n.classifiers import KNearestNeighbor

# 创建一个 kNN 分类器实例。
# 请记住，训练 kNN 分类器其实什么都不做：
# 分类器只是记住数据，不做进一步处理
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

现在我们希望用 kNN 分类器对测试数据进行分类。回顾一下，这个过程可以分为两个步骤：

1. 首先我们必须计算所有测试样本与所有训练样本之间的距离。
2. 给定这些距离，对于每个测试样本，我们找到 k 个最近的样本，让它们投票决定标签。

让我们先计算所有训练和测试样本之间的距离矩阵。例如，如果有 **Ntr** 个训练样本和 **Nte** 个测试样本，这一步应该得到一个 **Nte x Ntr** 的矩阵，其中每个元素 (i,j) 表示第 i 个测试样本和第 j 个训练样本之间的距离。

**注意：对于本笔记本要求你实现的三种距离计算方法，不能使用 numpy 提供的 np.linalg.norm() 函数。**

首先，打开 `cs231n/classifiers/k_nearest_neighbor.py`，实现 `compute_distances_two_loops` 函数，该函数使用双重循环遍历所有 (test, train) 样本对，并逐个计算距离矩阵的元素。

<details><summary>英文原文</summary>

We would now like to classify the test data with the kNN classifier. Recall that we can break down this process into two steps:

1. First we must compute the distances between all test examples and all train examples.
2. Given these distances, for each test example we find the k nearest examples and have them vote for the label

Lets begin with computing the distance matrix between all training and test examples. For example, if there are **Ntr** training examples and **Nte** test examples, this stage should result in a **Nte x Ntr** matrix where each element (i,j) is the distance between the i-th test and j-th train example.

**Note: For the three distance computations that we require you to implement in this notebook, you may not use the np.linalg.norm() function that numpy provides.**

First, open `cs231n/classifiers/k_nearest_neighbor.py` and implement the function `compute_distances_two_loops` that uses a (very inefficient) double loop over all pairs of (test, train) examples and computes the distance matrix one element at a time.

</details>

In [ ]:
# 打开 cs231n/classifiers/k_nearest_neighbor.py 并实现
# compute_distances_two_loops。

# 测试你的实现：
dists = classifier.compute_distances_two_loops(X_test)
print(dists.shape)

In [ ]:
# 我们可以可视化距离矩阵：每一行是一个测试样本，
# 以及它与训练样本的距离
plt.imshow(dists, interpolation='none')
plt.show()

**内嵌问题 1**

注意距离矩阵中的结构化模式，其中一些行或列明显更亮。（注意，默认配色方案下，黑色表示距离较小，白色表示距离较大。）

- 数据中导致明显亮行的原因是什么？
- 列的原因又是什么？

$\color{blue}{\textit 你的答案:}$ *请填写。*

<details><summary>英文原文</summary>

**Inline Question 1**

Notice the structured patterns in the distance matrix, where some rows or columns are visibly brighter. (Note that with the default color scheme black indicates low distances while white indicates high distances.)

- What in the data is the cause behind the distinctly bright rows?
- What causes the columns?

$\color{blue}{\textit Your Answer:}$ *fill this in.*

</details>

In [ ]:
# 现在实现 predict_labels 函数并运行下面的代码：
# 我们使用 k = 1（即最近邻）。
y_test_pred = classifier.predict_labels(dists, k=1)

# 计算并打印正确预测的样本比例
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

你应该能看到大约 `27%` 的准确率。现在让我们尝试更大的 `k`，比如 `k = 5`：

<details><summary>英文原文</summary>

You should expect to see approximately `27%` accuracy. Now lets try out a larger `k`, say `k = 5`:

</details>

In [ ]:
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

你应该能看到比 `k = 1` 略高的性能。

<details><summary>英文原文</summary>

You should expect to see a slightly better performance than with `k = 1`.

</details>

**内嵌问题 2**

我们也可以使用其他距离度量，比如 L1 距离。
对于某张图片 $I_k$ 在位置 $(i,j)$ 的像素值 $p_{ij}^{(k)}$，

所有图片所有像素的均值 $\mu$ 为 $$\mu=\frac{1}{nhw}\sum_{k=1}^n\sum_{i=1}^{h}\sum_{j=1}^{w}p_{ij}^{(k)}$$
所有图片在每个像素位置的均值 $\mu_{ij}$ 为
$$\mu_{ij}=\frac{1}{n}\sum_{k=1}^np_{ij}^{(k)}.$$
总体标准差 $\sigma$ 和像素点标准差 $\sigma_{ij}$ 定义类似。

下面哪些预处理步骤不会改变使用 L1 距离的最近邻分类器的性能？可多选。注意，训练和测试样本都做同样的预处理。

1. 减去均值 $\mu$ ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu$.)
2. 减去每个像素的均值 $\mu_{ij}$  ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu_{ij}$.)
3. 减去均值 $\mu$ 并除以标准差 $\sigma$。
4. 减去每个像素的均值 $\mu_{ij}$ 并除以每个像素的标准差 $\sigma_{ij}$。
5. 旋转数据的坐标轴，即所有图片旋转同一个角度。旋转导致的空白区域用同一个像素值填充，不做插值。

$\color{blue}{\textit 你的答案:}$


$\color{blue}{\textit 你的解释:}$

<details><summary>英文原文</summary>

**Inline Question 2**

We can also use other distance metrics such as L1 distance.
For pixel values $p_{ij}^{(k)}$ at location $(i,j)$ of some image $I_k$,

the mean $\mu$ across all pixels over all images is $$\mu=\frac{1}{nhw}\sum_{k=1}^n\sum_{i=1}^{h}\sum_{j=1}^{w}p_{ij}^{(k)}$$
And the pixel-wise mean $\mu_{ij}$ across all images is
$$\mu_{ij}=\frac{1}{n}\sum_{k=1}^np_{ij}^{(k)}.$$
The general standard deviation $\sigma$ and pixel-wise standard deviation $\sigma_{ij}$ is defined similarly.

Which of the following preprocessing steps will not change the performance of a Nearest Neighbor classifier that uses L1 distance? Select all that apply. To clarify, both training and test examples are preprocessed in the same way.

1. Subtracting the mean $\mu$ ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu$.)
2. Subtracting the per pixel mean $\mu_{ij}$  ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu_{ij}$.)
3. Subtracting the mean $\mu$ and dividing by the standard deviation $\sigma$.
4. Subtracting the pixel-wise mean $\mu_{ij}$ and dividing by the pixel-wise standard deviation $\sigma_{ij}$.
5. Rotating the coordinate axes of the data, which means rotating all the images by the same angle. Empty regions in the image caused by rotation are padded with a same pixel value and no interpolation is performed.

$\color{blue}{\textit Your Answer:}$


$\color{blue}{\textit Your Explanation:}$

</details>

In [ ]:
# 现在通过部分向量化加速距离矩阵计算，
# 用一个循环。实现 compute_distances_one_loop 函数并运行下面的代码：
dists_one = classifier.compute_distances_one_loop(X_test)

# 为了确保我们的向量化实现是正确的，我们要确保它
# 与朴素实现一致。有很多方法可以判断两个矩阵是否相似；
# 最简单之一是 Frobenius 范数。如果你没见过，Frobenius 范数就是所有元素差的平方和的平方根；
# 换句话说，把矩阵展平成向量，计算它们的欧氏距离。
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('One loop difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# 现在在 compute_distances_no_loops 里实现完全向量化版本，
# 并运行代码
dists_two = classifier.compute_distances_no_loops(X_test)

# 检查距离矩阵是否与之前计算的一致：
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('No loop difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# 我们来比较各实现的速度
def time_function(f, *args):
    """
    调用函数 f 并返回执行所需的时间（秒）。
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Two loop version took %f seconds' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('One loop version took %f seconds' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('No loop version took %f seconds' % no_loop_time)

# 你应该能看到完全向量化实现的性能显著提升！

# 注意：取决于你使用的机器，
# 从两层循环到一层循环可能看不到加速，
# 甚至可能更慢。

### 交叉验证

我们已经实现了 k-最近邻分类器，但 k 的值是随意设为 5 的。现在我们将通过交叉验证确定这个超参数的最佳值。

<details><summary>英文原文</summary>

### Cross-validation

We have implemented the k-Nearest Neighbor classifier but we set the value k = 5 arbitrarily. We will now determine the best value of this hyperparameter with cross-validation.

</details>

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
################################################################################
# TODO:                                                                        #
# 将训练数据分成若干折。分割后，X_train_folds 和 y_train_folds 应该都是长度为 num_folds 的列表， #
# 其中 y_train_folds[i] 是 X_train_folds[i] 中样本的标签向量。                     #
# 提示：查阅 numpy 的 array_split 函数。                                          #
#                                                                              #
# Split up the training data into folds. After splitting, X_train_folds and    #
# y_train_folds should each be lists of length num_folds, where                #
# y_train_folds[i] is the label vector for the points in X_train_folds[i].     #
# Hint: Look up the numpy array_split function.                                #
################################################################################


# 一个字典，用于保存不同 k 值下交叉验证得到的准确率
# 交叉验证后，k_to_accuracies[k] 应该是长度为 num_folds 的列表，保存每折的准确率。
# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation. After running cross-validation,
# k_to_accuracies[k] should be a list of length num_folds giving the different
# accuracy values that we found when using that value of k.
k_to_accuracies = {}


################################################################################
# TODO:                                                                        #
# 执行 k 折交叉验证以找到最佳 k。对于每个可能的 k，运行 kNN 算法 num_folds 次，         #
# 每次用所有折中的所有数据作为训练集，最后一折作为验证集。把所有折和所有 k 的准确率保存到字典 #
#                                                                              #
# Perform k-fold cross validation to find the best value of k. For each        #
# possible value of k, run the k-nearest-neighbor algorithm num_folds times,   #
# where in each case you use all but one of the folds as training data and the #
# last fold as a validation set. Store the accuracies for all fold and all     #
# values of k in the k_to_accuracies dictionary.                               #
################################################################################


# 打印计算得到的准确率
# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

In [ ]:
# 绘制原始观测点
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# 用误差条绘制趋势线，误差条对应标准差
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()

In [ ]:
# 根据上面的交叉验证结果，选择最佳 k，
# 用全部训练数据重新训练分类器，并在测试数据上测试。
# 你应该能在测试数据上得到超过 28% 的准确率。
best_k = 1

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred = classifier.predict(X_test, k=best_k)

# 计算并显示准确率
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

**内嵌问题 3**

关于 $k$-最近邻（$k$-NN） 分类器，下面哪些说法在分类场景下对所有 k 都成立？可多选。
1. k-NN 分类器的决策边界是线性的。
2. 1-NN 的训练误差总是小于或等于 5-NN。
3. 1-NN 的测试误差总是小于 5-NN。
4. 用 k-NN 分类器对测试样本分类所需时间会随着训练集规模增长。
5. 以上都不对。

$\color{blue}{\textit 你的答案:}$


$\color{blue}{\textit 你的解释:}$

<details><summary>英文原文</summary>

**Inline Question 3**

Which of the following statements about $k$-Nearest Neighbor ($k$-NN) are true in a classification setting, and for all $k$? Select all that apply.
1. The decision boundary of the k-NN classifier is linear.
2. The training error of a 1-NN will always be lower than or equal to that of 5-NN.
3. The test error of a 1-NN will always be lower than that of a 5-NN.
4. The time needed to classify a test example with the k-NN classifier grows with the size of the training set.
5. None of the above.

$\color{blue}{\textit Your Answer:}$


$\color{blue}{\textit Your Explanation:}$

</details>